### 第4週、3日目 - その他のランググラフへようこそ。

In [1]:
# import

# 基本
import os
import random
import requests
from dotenv import load_dotenv
from pydantic import BaseModel
from typing import Annotated

# langgraph
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode, tools_condition

# 表示
from IPython.display import Image, display
import gradio as gr

In [ ]:
# 私たちのお気に入りの最初のステップ！ところで、乗組員は私たちのためにこれをやっていた。
load_dotenv(override=True)


### まず、Langsmithをセットアップしましょう！

https://langsmith.com

### 次に、Langchainコミュニティの有用な機能を次に示します。

In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper

serper = GoogleSerperAPIWrapper()
serper.run("What is the capital of France?")

### ここに、関数をツールに変換するためのLangchainラッパークラスがあります

In [4]:
from langchain.agents import Tool

tool_search =Tool(
        name="search",
        func=serper.run,
        description="Useful for when you need more information from an online search"
    )



### これで、Langchain Wayというツールを試すことができます

In [ ]:
tool_search.invoke("What is the capital of France?")

### そして今、自分でツールを書きましょう

おなじみのものを選びます

In [6]:
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_user = os.getenv("PUSHOVER_USER")
pushover_url = "https://api.pushover.net/1/messages.json"

def push(text: str):
    """Send a push notification to the user"""
    requests.post(pushover_url, data = {"token": pushover_token, "user": pushover_user, "message": text})

In [7]:
tool_push = Tool(
        name="send_push_notification",
        func=push,
        description="useful for when you want to send a push notification"
    )

tool_push.invoke("Hello, me")

### 昨日からグラフに戻ります

1つの小さな変更 - 状態オブジェクトのベースモデルの代わりにtypeddictを使用する

ツールを実装するときは、常にコードに2つの変更を加える必要があります。

1。電話をかけるときにJSONでOpenAIにツールを提供するために変更

2。結果を処理するための変更：finish_reason == "tool_calls"を維持し、コールを取得し、関数を実行し、結果を提供するモデルを探します。

### それらをまとめてください

In [8]:
tools = [tool_search, tool_push]

In [9]:
# ステップ1：状態オブジェクトを定義します
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [11]:
# ステップ2：この状態クラスでグラフビルダーを起動します
graph_builder = StateGraph(State)

In [12]:
# これは異なります：

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

In [ ]:
# ステップ3：ノードを作成します


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

In [ ]:
# ステップ4：エッジを作成します


graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")

# ツールが呼び出されるときはいつでも、チャットボットに戻り、次のステップを決定します
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

In [ ]:
# ステップ5：グラフをコンパイルします
graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

### それでおしまい！そして、これをしましょう：

In [ ]:
def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]})
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()

## メモリを追加する時が来ました！

###でも待ってください！

このグラフ全体が州を維持し、州に適用しています。

なぜこの処理メモリがないのですか？

###これはランググラフを理解するための重要なポイントです

>スーパーステップは、グラフノード上の単一の反復と見なすことができます。並行して実行されるノードは同じスーパーステップの一部であり、実行されるノードはシーケンシャルに個別のスーパーステップに属します。


グラフの1つの「スーパーステップ」は、エージェント間の渡されたメッセージの1つの呼び出しを表しています。

等級ランググラフでは、スーパーステップごとにグラフを実行するためにInvokeを呼び出します。相互作用ごとに。

還元剤は、1つのスーパーステップ内で状態の更新を自動的に処理しますが、それらの間ではありません。

それがチェックポイントが達成することです。

In [2]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [ ]:
# ステップ1および2
graph_builder = StateGraph(State)


# ステップ3
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    print(state)
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

# ステップ4
graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

# ステップ5
graph = graph_builder.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "1"}}

def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()

In [ ]:
graph.get_state(config)

In [ ]:
# 最新の最初

list(graph.get_state_history(config))

### Langgraphは、状態を以前の時点に戻し、分岐するためのツールを提供します。

```
config = {"configurable": {"thread_id": "1", "checkpoint_id": ...}}
graph.invoke(None, config=config)
```

また、これにより、回復できる安定したシステムを構築し、以前のチェックポイントから再実行できます。

### 次に、SQLに保存しましょう

###これがランググラフの力です。

In [3]:
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

db_path = "memory.db"
conn = sqlite3.connect(db_path, check_same_thread=False)
sql_memory = SqliteSaver(conn)

In [ ]:
# ステップ1および2
graph_builder = StateGraph(State)


# ステップ3
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    print(state)
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

# ステップ4
graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

# ステップ5
graph = graph_builder.compile(checkpointer=sql_memory)
display(Image(graph.get_graph().draw_mermaid_png()))
 

In [ ]:
config = {"configurable": {"thread_id": "3"}}

def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()